# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={ODBC Driver 17 for SQL Server};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "UID=WOODFIELD\\jmeza@wdsrx.com;"
                        "PWD=Clavezaza2022;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

Date      id Warehouse  Shipments   Customer ShipmentStatus
0  Mar 15 2023  8:30AM  259319        10     HH9118  Hush Hush      Executing
1  Mar 15 2023  8:30AM  259317        10  PRF-55479    Bio-PRF      Executing
2  Mar 15 2023  8:30AM  259317        10  PRF-55483    Bio-PRF      Executing
3  Mar 15 2023  8:30AM  259317        10  PRF-55489    Bio-PRF      Executing
4  Mar 15 2023  8:30AM  259317        10  PRF-55492    Bio-PRF       Released
5  Mar 15 2023  8:30AM  259317        10  PRF-55504    Bio-PRF       Released
6  Mar 15 2023  8:30AM  259317        10  PRF-55510    Bio-PRF      Executing
7  Mar 15 2023  8:30AM  259317        10  PRF-55534    Bio-PRF      Executing
8  Mar 15 2023  8:30AM  259317        10  PRF-55538    Bio-PRF      Executing
9  Mar 15 2023  8:30AM  259317        10  PRF-55515    Bio-PRF      Executing

In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

id ShipmentStatus  Shipments
10  259317       Released          3
11  259318      Executing          1
12  259319      Executing          1
13  259320       Released          1
14  259334       Released          1

In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus  Completed  Executing  Released
id                                            
259317                NaN       11.0       3.0
259318                NaN        1.0       NaN
259319                NaN        1.0       NaN
259320                NaN        NaN       1.0
259334                NaN        NaN       1.0

In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus  Completed  Executing  Released
id                                            
259225                0.0        0.0       1.0
259258                0.0        0.0       1.0
259260               13.0        4.0       0.0
259311                0.0        0.0       1.0
259312                0.0        1.0       0.0

In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

ShipmentStatus  Completed  Executing  Released
id                                            
259225                  0          0         1
259258                  0          0         1
259260                 13          4         0
259311                  0          0         1
259312                  0          1         0

In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus      id  Completed  Executing  Released
0               259225          0          0         1
1               259258          0          0         1
2               259260         13          4         0
3               259311          0          0         1
4               259312          0          1         0

In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

ShipmentStatus      id Completed Executing Released
0               259225                            1
1               259258                            1
2               259260        13         4         
3               259311                            1
4               259312                   1

In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Completed    object
Executing    object
Released     object
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

Date      id Warehouse           Customer
0   Mar 15 2023  8:30AM  259319        10          Hush Hush
1   Mar 15 2023  8:30AM  259317        10            Bio-PRF
15  Mar 15 2023  8:30AM  259318        10     Citieffe, Inc.
16  Mar 15 2023  8:30AM  259315        10  Yusen – 3D Matrix
20  Mar 14 2023  4:37PM  259334        21   OK Capsule, Inc.
21  Mar 14 2023  3:45PM  259320        22  NBTY Global, Inc.
22  Mar 14 2023  3:16PM  259314        21  NBTY Global, Inc.
23  Mar 14 2023  3:10PM  259312        12          Hush Hush
24  Mar 14 2023  3:02PM  259311        19     ClearSpec, LLC
25  Mar 14 2023 10:32AM  259260        10   Eywa Pharma Inc.

In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

Date      id Warehouse           Customer Completed  \
0  Mar 15 2023  8:30AM  259319        10          Hush Hush             
1  Mar 15 2023  8:30AM  259317        10            Bio-PRF             
2  Mar 15 2023  8:30AM  259318        10     Citieffe, Inc.             
3  Mar 15 2023  8:30AM  259315        10  Yusen – 3D Matrix             
4  Mar 14 2023  4:37PM  259334        21   OK Capsule, Inc.             
5  Mar 14 2023  3:45PM  259320        22  NBTY Global, Inc.             
6  Mar 14 2023  3:16PM  259314        21  NBTY Global, Inc.             
7  Mar 14 2023  3:10PM  259312        12          Hush Hush             
8  Mar 14 2023  3:02PM  259311        19     ClearSpec, LLC             
9  Mar 14 2023 10:32AM  259260        10   Eywa Pharma Inc.        13   

  Executing Released  
0         1           
1        11        3  
2         1           
3         3        1  
4                  1  
5                  1  
6         1           
7         1           
8                  1  
9         4

In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

Date      id Warehouse           Customer Released  \
0  Mar 15 2023  8:30AM  259319        10          Hush Hush            
1  Mar 15 2023  8:30AM  259317        10            Bio-PRF        3   
2  Mar 15 2023  8:30AM  259318        10     Citieffe, Inc.            
3  Mar 15 2023  8:30AM  259315        10  Yusen – 3D Matrix        1   
4  Mar 14 2023  4:37PM  259334        21   OK Capsule, Inc.        1   
5  Mar 14 2023  3:45PM  259320        22  NBTY Global, Inc.        1   
6  Mar 14 2023  3:16PM  259314        21  NBTY Global, Inc.            
7  Mar 14 2023  3:10PM  259312        12          Hush Hush            
8  Mar 14 2023  3:02PM  259311        19     ClearSpec, LLC        1   
9  Mar 14 2023 10:32AM  259260        10   Eywa Pharma Inc.            

  Executing Completed  
0         1            
1        11            
2         1            
3         3            
4                      
5                      
6         1            
7         1            
8                      
9         4        13

In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

Date      id Warehouse           Customer Released  \
0  Mar 15 2023  8:30AM  259319        10          Hush Hush            
1  Mar 15 2023  8:30AM  259317        10            Bio-PRF        3   
2  Mar 15 2023  8:30AM  259318        10     Citieffe, Inc.            
3  Mar 15 2023  8:30AM  259315        10  Yusen – 3D Matrix        1   
4  Mar 14 2023  4:37PM  259334        21   OK Capsule, Inc.        1   

  Executing Completed  
0         1            
1        11            
2         1            
3         3            
4

In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

Date      id Warehouse           Customer  Released  \
0  Mar 15 2023  8:30AM  259319        10          Hush Hush       NaN   
1  Mar 15 2023  8:30AM  259317        10            Bio-PRF       3.0   
2  Mar 15 2023  8:30AM  259318        10     Citieffe, Inc.       NaN   
3  Mar 15 2023  8:30AM  259315        10  Yusen – 3D Matrix       1.0   
4  Mar 14 2023  4:37PM  259334        21   OK Capsule, Inc.       1.0   

   Executing  Completed  
0        1.0        NaN  
1       11.0        NaN  
2        1.0        NaN  
3        3.0        NaN  
4        NaN        NaN

In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

Date      id Warehouse           Customer  Released  \
0  Mar 15 2023  8:30AM  259319        10          Hush Hush       0.0   
1  Mar 15 2023  8:30AM  259317        10            Bio-PRF       3.0   
2  Mar 15 2023  8:30AM  259318        10     Citieffe, Inc.       0.0   
3  Mar 15 2023  8:30AM  259315        10  Yusen – 3D Matrix       1.0   
4  Mar 14 2023  4:37PM  259334        21   OK Capsule, Inc.       1.0   

   Executing  Completed  
0        1.0        0.0  
1       11.0        0.0  
2        1.0        0.0  
3        3.0        0.0  
4        0.0        0.0

In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

id  Released  Executing  Completed
Warehouse                                         
10         1296529       4.0       20.0       13.0
12          259312       0.0        1.0        0.0
19          518569       2.0        0.0        0.0
20          259225       1.0        0.0        0.0
21          518648       1.0        1.0        0.0
22          259320       1.0        0.0        0.0

In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

Warehouse       id  Released  Executing  Completed
0        10  1296529       4.0       20.0       13.0
1        12   259312       0.0        1.0        0.0
2        19   518569       2.0        0.0        0.0
3        20   259225       1.0        0.0        0.0
4        21   518648       1.0        1.0        0.0
5        22   259320       1.0        0.0        0.0

In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

Warehouse  Released  Executing  Completed
0        10       4.0       20.0       13.0
1        12       0.0        1.0        0.0
2        19       2.0        0.0        0.0
3        20       1.0        0.0        0.0
4        21       1.0        1.0        0.0
5        22       1.0        0.0        0.0

In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

Warehouse    Status  Value
0        10  Released    4.0
1        12  Released    0.0
2        19  Released    2.0
3        20  Released    1.0
4        21  Released    1.0

In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse    10   12   19   20   21   22
Status                                  
Completed  13.0  0.0  0.0  0.0  0.0  0.0
Executing  20.0  1.0  0.0  0.0  1.0  0.0
Released    4.0  0.0  2.0  1.0  1.0  1.0

In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse     Status    10   12   19   20   21   22
0          Completed  13.0  0.0  0.0  0.0  0.0  0.0
1          Executing  20.0  1.0  0.0  0.0  1.0  0.0
2           Released   4.0  0.0  2.0  1.0  1.0  1.0

In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

Status    10   12   19   20   21   22
0  Completed  13.0  0.0  0.0  0.0  0.0  0.0
1  Executing  20.0  1.0  0.0  0.0  1.0  0.0
2   Released   4.0  0.0  2.0  1.0  1.0  1.0

In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()